<a href="https://colab.research.google.com/github/pradyunramesh/MachineLearningProject562/blob/main/Comp562_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Extend to multi-class logistic regression:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving subjects-questions.csv to subjects-questions (1).csv


In [ ]:
subjects_data = pd.read_csv("subjects-questions.csv")
subjects_data.head(5)

,eng,Subject
0,An anti-forest measure is\nA. Afforestation\nB...,Biology
1,"Among the following organic acids, the acid pr...",Chemistry
2,If the area of two similar triangles are equal...,Maths
3,"In recent year, there has been a growing\nconc...",Biology
4,Which of the following statement\nregarding tr...,Physics


In [ ]:
subjects_data.isnull().sum()

eng        0
Subject    0
dtype: int64

In [ ]:
#Apart from Biology, all other classes appear to be of similar size
subjects_data.groupby(['Subject']).size()

Subject
Biology      13124
Chemistry    37767
Maths        33190
Physics      38438
dtype: int64

In [ ]:
subjects_data.loc[subjects_data["Subject"] == "Biology", "Subject"] = 0
subjects_data.loc[subjects_data["Subject"] == "Chemistry", "Subject"] = 1
subjects_data.loc[subjects_data["Subject"] == "Physics", "Subject"] = 2
subjects_data.loc[subjects_data["Subject"] == "Maths", "Subject"] = 3

In [ ]:
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
X_vals = subjects_data['eng'].to_numpy()
X_vals

array(['An anti-forest measure is\nA. Afforestation\nB. Selective grazing\nC . Clearing forest\nD. Selective felling',
       'Among the following organic acids, the acid present in rancid butter is:\nA . pyruvic acid\nB. lactic acid\nc. butyric acid\nD. acetic acid',
       'If the area of two similar triangles are equal, then they are\nA . equilateral\nB. isosceles\nc. congruent\nD. not congruent',
       ...,
       'Member of Dipnoi are native of\nA. India.\nB. Africa and Australia.\nc. New Zealand\nD. None of the above',
       'In one average-life,\nA. half the active nuclei decay\nB. less than half the active nuclei decay.\nC. more than half the active nuclei decay.\nD. all the nuclei decay.',
       'State whether true or false.\nA magnetic field is a region around a magnet where magnetic lines of force\nact\nA. True\nB. False'],
      dtype=object)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_vals_multiple = np.array(subjects_data['Subject'],dtype=np.float32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_initial, X_test, y_train_initial, y_test = train_test_split(X_vals, y_vals_multiple, test_size=0.2, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train_initial, y_train_initial, test_size = 0.2, random_state = 42)

In [ ]:
y_train_multiple = to_categorical(y_train)
y_test_multiple = to_categorical(y_test)
y_validation_multiple = to_categorical(y_validation)

In [ ]:
embeddings_train = model.encode(X_train)
embeddings_train

array([[-0.18480662, -0.02374146,  0.0423011 , ..., -0.43142626,
         0.6468544 ,  0.38509923],
       [ 0.37487864,  0.12404861, -0.08612633, ...,  0.06217834,
        -0.25348204,  0.00627198],
       [ 0.02346818,  0.25391117, -0.11395633, ...,  0.27049348,
         0.00884567,  0.1540007 ],
       ...,
       [-0.29296255,  0.6057741 ,  0.23564363, ..., -0.33942014,
        -0.3326536 ,  0.12453619],
       [-0.05065663,  0.29535618,  0.3795538 , ..., -0.28573978,
         0.3474969 ,  0.25256142],
       [ 0.05757226,  0.15701616,  0.39181322, ..., -0.28199792,
         0.08951461, -0.17346118]], dtype=float32)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
embeddings_train_new = scaler.fit_transform(embeddings_train)

In [ ]:
embeddings_test = model.encode(X_test)
embeddings_test_new = scaler.fit_transform(embeddings_test)
embeddings_validation = model.encode(X_validation)
embeddings_validation_new = scaler.fit_transform(embeddings_validation)

In [ ]:
#Baseline machine learning models:
#1) Logistic Regression model:
from sklearn.linear_model import LogisticRegression
reg_model = LogisticRegression(max_iter = 500)
result = reg_model.fit(embeddings_train, y_train)
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(reg_model.score(embeddings_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.9002


In [ ]:
#SVM Model with polynomial kernel:
from sklearn.metrics.pairwise import kernel_metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn import svm
poly_kernel = svm.SVC(kernel = 'poly', degree = 3)
rbf_kernel = svm.SVC(kernel = 'rbf')
poly_kernel.fit(embeddings_train, y_train)
y_pred_poly = poly_kernel.predict(embeddings_test)
print('Accuracy of SVM with polynomial kernel on test set: {:.4f}'.format(poly_kernel.score(embeddings_test, y_test)))
#print('F1 Score: %.3f' % f1_score(y_test, y_pred_poly))

Accuracy of SVM with polynomial kernel on test set: 0.9382


In [ ]:
#SVM Model with rbf kernel:
rbf_kernel.fit(embeddings_train, y_train)
y_pred_rbf = rbf_kernel.predict(embeddings_test)
print('Accuracy of SVM with rbf kernel on test set: {:.4f}'.format(rbf_kernel.score(embeddings_test, y_test)))
#print('F1 Score: %.3f' % f1_score(y_test, y_pred_rbf))

Accuracy of SVM with rbf kernel on test set: 0.9346


In [ ]:
#Basic neural network for text classification
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
tf.random.set_seed(42)
model_multiple = Sequential()
model_multiple.add(Flatten()) #Flattens the multi-dimensional input tensor into single dimension
model_multiple.add(Dense(384, activation = 'relu'))
model_multiple.add(Dropout(rate = 0.5)) #Dropout rate selected by varying hyperparameters
model_multiple.add(Dense(128, activation = 'relu'))
model_multiple.add(Dropout(rate = 0.5)) #Dropout rate selected by varying hyperparameters
model_multiple.add(Dense(4,activation = 'sigmoid'))
model_multiple.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
tensor_input_multiple = tf.convert_to_tensor(embeddings_train)
fit2 = model_multiple.fit(tensor_input_multiple, y_train_multiple, validation_data = (embeddings_validation, y_validation_multiple), epochs = 5, batch_size = 20)

Epoch 1/5
3921/3921 [==============================] - 20s 5ms/step - loss: 0.1579 - accuracy: 0.8833 - val_loss: 0.1180 - val_accuracy: 0.9096
Epoch 2/5
3921/3921 [==============================] - 15s 4ms/step - loss: 0.1254 - accuracy: 0.9059 - val_loss: 0.1103 - val_accuracy: 0.9138
Epoch 3/5
3921/3921 [==============================] - 14s 4ms/step - loss: 0.1150 - accuracy: 0.9157 - val_loss: 0.1026 - val_accuracy: 0.9213
Epoch 4/5
3921/3921 [==============================] - 15s 4ms/step - loss: 0.1068 - accuracy: 0.9209 - val_loss: 0.1003 - val_accuracy: 0.9230
Epoch 5/5
3921/3921 [==============================] - 15s 4ms/step - loss: 0.1008 - accuracy: 0.9242 - val_loss: 0.0979 - val_accuracy: 0.9250


In [ ]:
model_multiple.evaluate(embeddings_test, y_test_multiple)

766/766 [==============================] - 2s 2ms/step - loss: 0.1002 - accuracy: 0.9246


[0.10016343742609024, 0.9245837330818176]

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab = 5000
X_train_cnn = [one_hot(data, vocab,filters='!"%&()./:;?[\]^_',lower=True, split=' ') for data in X_train]
X_test_cnn = [one_hot(data, vocab,filters='!"%&()./:;?[\]^_',lower=True, split=' ') for data in X_test]
X_validation_cnn = [one_hot(data, vocab,filters='!"%&()./:;?[\]^_',lower=True, split=' ') for data in X_validation]
X_train_pad_cnn = pad_sequences(X_train_cnn, maxlen= 500, padding='post')
X_test_pad_cnn = pad_sequences(X_test_cnn, maxlen=500, padding='post')
X_validation_pad_cnn = pad_sequences(X_validation_cnn, maxlen = 500, padding = 'post')

In [ ]:
# Ensemble CNN-GRU neural network for text classification
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv1D
from tensorflow.keras.layers import Bidirectional, GRU, Embedding, GlobalMaxPooling1D, MaxPooling1D, Flatten
model_cnn = Sequential()
model_cnn.add(Embedding(vocab,10, input_length = 500))
model_cnn.add(Conv1D(filters = 384, kernel_size = 3, activation = 'relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(128, activation = 'relu'))
model_cnn.add(Dropout(rate = 0.5))
model_cnn.add(Dense(4, activation = 'relu'))
model_cnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_cnn.fit(X_train_pad_cnn, y_train_multiple, validation_data = (X_validation_pad_cnn, y_validation_multiple), epochs = 5, batch_size = 20)
output1 = model_cnn.predict(X_train_pad_cnn)

Epoch 1/5
3921/3921 [==============================] - 33s 7ms/step - loss: 0.4039 - accuracy: 0.7007 - val_loss: 0.3169 - val_accuracy: 0.7999
Epoch 2/5
3921/3921 [==============================] - 32s 8ms/step - loss: 0.3052 - accuracy: 0.8131 - val_loss: 0.2751 - val_accuracy: 0.8499
Epoch 3/5
3921/3921 [==============================] - 23s 6ms/step - loss: 0.2925 - accuracy: 0.8229 - val_loss: 0.2823 - val_accuracy: 0.8386
Epoch 4/5
3921/3921 [==============================] - 24s 6ms/step - loss: 0.2989 - accuracy: 0.8287 - val_loss: 0.3692 - val_accuracy: 0.8505
Epoch 5/5
2451/2451 [==============================] - 6s 2ms/step


In [ ]:
model_cnn.evaluate(X_test_pad_cnn, y_test_multiple)

766/766 [==============================] - 3s 4ms/step - loss: 0.2790 - accuracy: 0.8349


[0.27902671694755554, 0.8349249362945557]

In [ ]:
model_norm = Sequential()
model_norm.add(Embedding(vocab, 10, input_length = 500))
model_norm.add(Bidirectional(tf.keras.layers.GRU(384)))
model_norm.add(Dense(384, activation = 'relu'))
model_norm.add(Dropout(rate = 0.7))
model_norm.add(Dense(128, activation = 'relu'))
model_norm.add(Dropout(rate = 0.7))
model_norm.add(Dense(4, activation = 'sigmoid'))
model_norm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_norm.fit(X_train_pad_cnn, y_train_multiple, validation_data = (X_validation_pad_cnn, y_validation_multiple), epochs = 4, batch_size = 20)
output2 = model_norm.predict(X_train_pad_cnn)

Epoch 1/4
3921/3921 [==============================] - 238s 60ms/step - loss: 0.2816 - accuracy: 0.7517 - val_loss: 0.1957 - val_accuracy: 0.8423
Epoch 2/4
3921/3921 [==============================] - 232s 59ms/step - loss: 0.1800 - accuracy: 0.8642 - val_loss: 0.1624 - val_accuracy: 0.8759
Epoch 3/4
3921/3921 [==============================] - 231s 59ms/step - loss: 0.1461 - accuracy: 0.8942 - val_loss: 0.1597 - val_accuracy: 0.8747
Epoch 4/4
2451/2451 [==============================] - 58s 24ms/step


In [ ]:
model_norm.evaluate(X_test_pad_cnn, y_test_multiple)

766/766 [==============================] - 19s 25ms/step - loss: 0.1677 - accuracy: 0.8831


[0.1677383929491043, 0.8830803036689758]

In [ ]:
output_test_1 = model_cnn.predict(X_test_pad_cnn)
output_test_2 = model_norm.predict(X_test_pad_cnn)
merged_outputs_test = np.concatenate([output_test_1, output_test_2], axis = 1)
output_validation_1 = model_cnn.predict(X_validation_pad_cnn)
output_validation_2 = model_norm.predict(X_validation_pad_cnn)
merged_outputs_validation = np.concatenate([output_validation_1, output_validation_2], axis = 1)

613/613 [==============================] - 14s 23ms/step


In [ ]:
merged_outputs = np.concatenate([output1, output2], axis = 1)
model_ensemble2 = Sequential()
model_ensemble2.add(Dense(4, activation = 'relu'))
model_ensemble2.add(Dense(4, activation = 'sigmoid'))
model_ensemble2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_ensemble2.fit(merged_outputs, y_train_multiple, validation_data = (merged_outputs_validation, y_validation_multiple), epochs = 5, batch_size = 20)

Epoch 1/5
3921/3921 [==============================] - 16s 4ms/step - loss: 0.1883 - accuracy: 0.8630 - val_loss: 0.1546 - val_accuracy: 0.8899
Epoch 2/5
3921/3921 [==============================] - 12s 3ms/step - loss: 0.0960 - accuracy: 0.9325 - val_loss: 0.1562 - val_accuracy: 0.8893
Epoch 3/5
3921/3921 [==============================] - 14s 4ms/step - loss: 0.0952 - accuracy: 0.9326 - val_loss: 0.1564 - val_accuracy: 0.8890
Epoch 4/5
3921/3921 [==============================] - 17s 4ms/step - loss: 0.0950 - accuracy: 0.9328 - val_loss: 0.1581 - val_accuracy: 0.8893
Epoch 5/5
3921/3921 [==============================] - 14s 4ms/step - loss: 0.0949 - accuracy: 0.9328 - val_loss: 0.1564 - val_accuracy: 0.8895


In [ ]:
model_ensemble2.evaluate(merged_outputs_test, y_test_multiple)

766/766 [==============================] - 2s 2ms/step - loss: 0.1589 - accuracy: 0.8865


[0.1589173972606659, 0.8865083456039429]